In [1]:
import re
import random
import numpy as np
from nltk.stem.lancaster import LancasterStemmer

stemmer = LancasterStemmer()

In [2]:
def _revenue_response(data:dict):
    return_list = dict()
    ratio_period = stemmer.stem(ratio["period"])

    if ratio["sales"] > 0:
        ratio_calc = f"greater than {f'last {ratio_period}' if ratio_period != 'yesterday' else f'{ratio_period}'}"
    elif ratio["sales"] == 0:
        ratio_calc = f"almost equivalent {f'last {ratio_period}' if ratio_period != 'yesterday' else f'{ratio_period}'}"
    else:
        ratio_calc = f"less than {f'last {ratio_period}' if ratio_period != 'yesterday' else f'{ratio_period}'}"
        
    sentiment = "gladly" if ratio["sales"] > 0 else "sadly"
    sentiment = sentiment if ratio["sales"] != 0 else ""
    ratio_times = f"{round(ratio['sales'], 3)} times" if ratio["sales"] != 0 else ""
    
    extra_responses = [
        f"_p _f2 Looking at the income so far, compared to {f'last {ratio_period}' if ratio_period != 'yesterday' else f'{ratio_period}'} ratio, {sentiment} we're {ratio_times} {ratio_calc} p_".replace("  ", " "),
        f"{'_p _f2 Today seems to be a good day _p' if ratio['sales'] > 0 else f'Business today is kind of slow compared to {ratio_period}'}"]
    
    extra_responses = np.random.choice([random.choice(extra_responses), None], p=[0.45, 0.55])
    
    data["responses"] = random.choice(data['responses'])
    amount = f"{data['currency_symbol']}{data['amount']}"
    
    
    data["responses"] = data["responses"].replace("_value_", amount).replace("  ", " ")
    if data["amount"] > 0:
        if extra_responses is not None:
            return_list["responses"] = [data["responses"], extra_responses]
        else:
            return_list["responses"] = data["responses"]
            
    elif data["amount"] < 0:
        return_list["responses"] = random.choice(["_h1 _f1 Sadly we are on a negative revenue h1_",
                                                 f"_h1 _f1 Unfortunately we are on a negative revenue of about {amount} h1_",
                                                 f"_h1 _f1 Still at {amount} but all figers cross for a great day h1_"])
        
    else:
        return_list["responses"] = random.choice(["_h1 _f1 Sorry we haven't made any sale nor service yet h1_",
                                                  "_h1 _f1 We haven't made any transaction yet but the day is still fresh looking forward to an exciting day h1_",
                                                  f"_h1 _f1 Still at {amount} but all figers cross for a great day h1_"])
        
    return_list["title"] = data["tag"].replace("_", " ")
    return_list["type"] = data["tag"].split("_")[-1]
    
    if data["queue"]:
        return_list["queue"] = data["queue"]
    if data["context"]:
        return_list["context"] = data["context"]
    
    return return_list


In [3]:
def _margin_response(data: dict):
    return_list = dict()
    
    data['responses'] = random.choice(data['responses'])
    amount = f"{data['currency_symbol']}{data['amount']}"
    
    extra_responses = [f"_p _f2 Looking forward to a great day _p"]
    extra_responses = np.random.choice([random.choice(extra_responses), None], p=[0.45, 0.55])
    
    if data["amount"] > 0:
        return_list["responses"] = data["responses"]
        
    elif data["amount"] < 0:
        return_list["responses"] = random.choice([
            "_h1 _f1 We are currently working on a negative profit of _value_ h1_",
            "_h1 _f1 Sadly we are on the bad side of business today of about _value_ margin h1_",
            "_h1 _f1 Unfortunately we are _value_ margin short today h1_"
        ])
        
    else:
        return_list["responses"] = random.choice([
            "_h1 _f1 We are still to make profit today h1_",
            "_h1 _f1 No benefitial transaction made yet, still at _value_ profit h1_",
            "_h1 _f1 Unfortunately we are still at _value_ margin now h1_"
        ])
    
    
    return_list["responses"] = return_list["responses"].replace("_value_", amount).replace("  ", " ")
    if extra_responses is not None:
        new_response = list([return_list["responses"], extra_responses])
        return_list["responses"] = new_response
    
    return_list["title"] = data["tag"].replace("_", " ")
    return_list["type"] = data["tag"].split("_")[-1]
    
    if data["queue"]:
        return_list["queue"] = data["queue"]
    if data["context"]:
        return_list["context"] = data["context"]
        
    return return_list
    

In [4]:
def _forecast_response(data: dict):
    return_list = {}
    data["responses"] = random.choice(data["responses"])
    amount = f"{data['currency_symbol']}{data['amount']}"
    
    get_period = {
        "daily": "today",
        "weekly": "this week",
        "monthly": "this month"
    }
    
    # if model accuracy is less than 69%
    if data["model_accuracy"] < 0.69:
        negative_response = np.random.choice(["_p _f2 Please contact universe customer service at appstechlabs@gmail.com because we care", None], p=[0.58, 0.42])
        if negative_response is not None:
            return_list["responses"] = [random.choice(["_h1 _f1 No forecast avaiable h1_",
                                                     f"_h1 _f1 Oh no something went wrong can't seem to forecast any revenue for {get_period.get(data['tag'].split('_')[0])} h1_",
                                                     f"_h1 _f1 I'm having trouble forecasting your revenue for {get_period.get(data['tag'].split('_')[0])} h1_"]), negative_response]
        
        else:
            return_list["responses"] = random.choice(["_h1 _f1 No forecast avaiable h1_",
                                                     f"_h1 _f1 Oh no something went wrong can't seem to forecast any revenue for {get_period.get(data['tag'].split('_')[0])} h1_",
                                                     f"_h1 _f1 I'm having trouble forecasting your revenue for {get_period.get(data['tag'].split('_')[0])} h1_"])
            return return_list

    elif data["amount"] == 0:
        return_list["responses"] = random.choice([f"_h1 _f1 Your forecast revenue for {get_period.get(data['tag'].split('_')[0])} is _value_. Not sure if {get_period.get(data['tag'].split('_')[0])} is a business day h1_",
                                                   f"_h1 _f1 I forecasted a revenue of _value_ for {get_period.get(data['tag'].split('_')[0])} surely {get_period.get(data['tag'].split('_')[0])} is not a business day h1_"
                                                   ])
        
        return_list["responses"] = return_list["responses"].replace("_value_", f"{amount}").replace("  ", " ")
    
    else:
        extra_responses = np.random.choice([f"_p _f2 I'm {data['model_accuracy'] * 100}% accurate about this forecast p_", None], p=[0.335, 0.665])
        if extra_responses is not None:
            return_list["responses"] = [data["responses"].replace("_value_", amount).replace("  ", " "), extra_responses]
        else:
            return_list["responses"] = data["responses"].replace("_value_", amount).replace("  ", " ")
    
    return_list["title"] = data["tag"].replace("_", " ")
    return_list["type"] = data["tag"].split("_")[-1]
    
    if data["queue"]:
        return_list["queue"] = data["queue"]
    if data["context"]:
        return_list["context"] = data["context"]
        
    return return_list


In [9]:
model_response = {
    'tag': 'today_revenue', 'queue': [], 'context': '',
    'responses': ["_h1 _f1 Here is your revenue for today _value_ h1_",
                  "_h1 _f1 Your revenue today is _value_ h1_",
                  "_h1 _f1 Currently your are at _value_ today h1_",
                  "_h1 _f1 _value_ h1_"
        ]
    }

revenue_data = {
    **model_response,
#     "amount": -898,
    "amount": 7898.8,
#     "amount": 0,
    "currency_symbol": "$"
}

ratio = {
    "cost": -0.8284145504461222,
    "current_period": {
          "end": "2020-08-31",
          "start": "2020-08-01"
    },
    "discount": -0.8285117220662183,
    "margin": -1.0,
    "period": "yesterday",
    "previous_period": {
          "end": "2020-07-31",
          "start": "2020-07-01"
    },
#     "sales": 4.212080957915905,
    "sales": 0,
    "tax": 32.32956728053327
}

model_response = {
    'tag': 'today_margin',
    'queue': [], 'context': '',
    'responses': ['_h1 _f1 Here is your margin for today _value_ h1_',
                  '_h1 _f1 _value_ is your profit for today h1_',
                  '_h1 _f1 Currently your are at. _value_ gain today h1_'
                 ]
        }

margin_data = {
    **model_response,
#     "amount": -898,
    "amount": 7428.0,
#     "amount": 0,
    "currency_symbol": "$"
}

model_response = {
    'tag': 'daily_forecast', 'queue': [], 'context': '',
    'responses': [
        '_h1 _f1 I forecasted a revenue of _value_ today h1_',
        '_h1 _f1 Your forecast revenue for today is _value_ h1_',
        '_h1 _f1 _value_ _h1'
    ]
}

forecast_data = {
    **model_response,
    "amount": 437.87,
    "model_accuracy": 0.870,
    "currency_symbol": "$"
}


def get_responses(responses):
    if isinstance(responses, list):
        return_list = {}
        for res in responses:
            return_list[f"{res['type']}"] = {
                "title": res["title"],
                "responses": res['responses'],
                "type": res["type"]
            }
            
        return return_list
    
    else:
        return responses
    
    
    
    
responses = [_revenue_response(revenue_data), _margin_response(margin_data), _forecast_response(forecast_data)]
# responses = _revenue_response(revenue_data)
responses = get_responses(responses)
responses

{'revenue': {'title': 'today revenue',
  'responses': '_h1 _f1 Your revenue today is $7898.8 h1_',
  'type': 'revenue'},
 'margin': {'title': 'today margin',
  'responses': ['_h1 _f1 Here is your margin for today $7428.0 h1_',
   '_p _f2 Looking forward to a great day _p'],
  'type': 'margin'},
 'forecast': {'title': 'daily forecast',
  'responses': '_h1 _f1 $437.87 _h1',
  'type': 'forecast'}}